In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src import (
    config,
    data,
    utils)

import src.config
import os
import pandas as pd
import torch

In [3]:
FILENAME = src.config.dataset_name
ROOT = utils.get_project_root_path()
VERBOSE = config.VERBOSE
RAW_PATH = src.config.raw_dataset_path
PROCESSED_PATH = src.config.processed_dataset_path

In [4]:
df_data = src.data.process(src.data.parse_file(ROOT + '/data/raw/' + FILENAME))

In [5]:
df_data

,Uniprot_AC,Kingdom,Type,Partition_No,Sequence,Label
0,Q8TF40,EUKARYA,NO_SP,0,<AA2fold> M A P T L F Q K L F S K R T G L G A ...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
1,Q1ENB6,EUKARYA,NO_SP,1,<AA2fold> M D F T S L E T T T F E E V V I A L ...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
2,P36001,EUKARYA,NO_SP,2,<AA2fold> M D D I S G R Q T L P R I N R L L E ...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
3,P55317,EUKARYA,NO_SP,0,<AA2fold> M L G T V K M E G H E T S D W N S Y ...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
4,P35583,EUKARYA,NO_SP,0,<AA2fold> M L G A V K M E G H E P S D W S S Y ...,IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...
...,...,...,...,...,...,...
20285,P0AB09,NEGATIVE,TAT,0,<AA2fold> M D K F D A N R R K L L A L G G V A ...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOOOOOOO...
20286,P0A5E2,POSITIVE,TAT,2,<AA2fold> M T T M I T L R R R F A V A V A G V ...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOOOOOOOOOO...
20287,P9WQP3,POSITIVE,TAT,1,<AA2fold> M Q L V D R V R G A V T G M S R R L ...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOO...
20288,P9WQP2,POSITIVE,TAT,1,<AA2fold> M Q L V D R V R G A V T G M S R R L ...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTOOO...


In [6]:
df_data['Partition_No'].value_counts()

Partition_No
1    7392
2    7002
0    5896
Name: count, dtype: int64

In [7]:
df_data['Type'].value_counts()

Type
NO_SP      15625
SP          2582
LIPO        1615
TAT          365
PILIN         70
TATLIPO       33
Name: count, dtype: int64

In [ ]:
x = df_data[df_data['Type'] == 'PILIN']
unique_chars = pd.DataFrame(list(''.join(x['Label'].to_list()))).value_counts()
unique_chars


In [1]:
import numpy as np
from sklearn.metrics import matthews_corrcoef, recall_score, precision_score, confusion_matrix, f1_score

In [2]:
def compute_mcc(true_labels: np.ndarray, pred_labels: np.ndarray, label_positive=1):
    """Compute MCC. binarize to true/false, according to label specified.
    For MCC1, subset data before calling this function"""
    # binarize everything
    true_labels = [1 if x == label_positive else 0 for x in true_labels]
    pred_labels = [1 if x == label_positive else 0 for x in pred_labels]

    return matthews_corrcoef(true_labels, pred_labels)

In [4]:
print(compute_mcc(np.array([1, 1, 1, 1, 1, 1]), np.array([1, 1, 1, 1, 1, 1]), 1))
print(compute_mcc(np.array([0, 0, 0, 0, 1, 1]), np.array([1, 1, 1, 1, 0, 0]), 1))
print(compute_mcc(np.array([1, 1, 1, 1, 0, 0]), np.array([1, 1, 1, 1, 0, 0]), 1))
print(compute_mcc(np.array([1, 1, 1, 1, 1, 0]), np.array([1, 1, 1, 1, 0, 0]), 1))

0.0
-1.0
1.0
0.6324555320336759


/Users/finnlueth/Developer/gits/SignalGPT/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [44]:
print(matthews_corrcoef(
    y_pred=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1]),
    y_true=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    ))
print(matthews_corrcoef(
    y_pred=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]),
    y_true=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])
    ))
print(matthews_corrcoef(
    y_pred=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    y_true=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])
    ))

0.0
0.35043832202523123
0.0


In [53]:
print(matthews_corrcoef(
    y_pred=np.array([x for x in "IIIIIIIIIIIIIIMMMMMMMMMMMMMMMMOOOOOOOOOOOOOOOOMMMMMMMMMMMMMMMMMIIIIIII"]),
    y_true=np.array([x for x in "IIIIIIIIIIIIIIMMMMMMMMMMMMMMMMOOOOOOOOOOOOOOOOMMMMMMMMMMMMMMMMMIIIIIII"])
    ))
print(matthews_corrcoef(
    y_pred=np.array([x for x in "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"]),
    y_true=np.array([x for x in "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"])
    ))


1.0
0.0


/Users/finnlueth/Developer/gits/SignalGPT/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
import src.model

In [ ]:
# truth = [x for x in "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIOIIIOIIII"]
# preds = [x for x in "IIIIIIIIIIIIIIIIIIIIIIIIIIIIOIIIIIIIIOIIIIIIII"]
truth = [x for x in "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"]
preds = [x for x in "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"]
print(*truth)
print(*preds)
enc_truth = src.model.one_vs_all_encoding(truth, 'I')
enc_preds = src.model.one_vs_all_encoding(preds, 'I')
print(*enc_truth)
print(*enc_preds)

In [ ]:
print(compute_mcc(enc_truth, enc_preds))

In [ ]:
confusion_matrix(enc_truth, enc_preds, labels=[0, 1])

---

In [8]:
# Didn't even need this :(
def remove_adjacent_duplicates(s):
    stack = []
    for i, c in enumerate(s):
        if not i+1 == len(s) and s[i+1] == c:
            pass
        else:
            stack.append(c)
    return ''.join(stack)

In [9]:
res = df_data['Label'].apply(lambda x: remove_adjacent_duplicates(x))

In [10]:
res.unique()

array(['I', 'SO', 'SOMI', 'SOM', 'OMIM', 'IMO', 'OMI', 'IM', 'IMOMIM',
       'IMOM', 'OM', 'IMOMI', 'OMIMO', 'OMIMOM', 'TO', 'LO', 'LOMI',
       'LOM', 'LOMIM', 'LOMIMO', 'SOMIM', 'SOMIMOM', 'SOMIMOMI', 'SOMIMO',
       'IMOMIMO', 'POMO', 'OMIMOMI'], dtype=object)

In [13]:
start_of_seq_tags = {x[0] for x in df_data['Label'].unique()}
end_of_seq_tags = {x[-1] for x in df_data['Label'].unique()}
start_of_seq_tags, end_of_seq_tags

({'I', 'L', 'O', 'P', 'S', 'T'}, {'I', 'M', 'O'})

In [25]:

def get_transitions_list(df):
    words = df['Label'].unique()
    transitions = set()
    
    for word in words:
        for i in range(len(word) - 1):
            transitions.add((word[i], word[i+1]))

    transitions_list = sorted(list(transitions))
    
    start_of_seq_tags = sorted(list({x[0] for x in df['Label'].unique()}))
    end_of_seq_tags = sorted(list({x[-1] for x in df['Label'].unique()}))
    
    for x in start_of_seq_tags:
        transitions_list.append(('<START>', x))
    for x in end_of_seq_tags:
        transitions_list.append((x, '<END>'))

    return transitions_list

transitions_list = get_transitions_list(df_data)
print(transitions_list)

[('I', 'I'), ('I', 'M'), ('L', 'L'), ('L', 'O'), ('M', 'I'), ('M', 'M'), ('M', 'O'), ('O', 'M'), ('O', 'O'), ('P', 'O'), ('P', 'P'), ('S', 'O'), ('S', 'S'), ('T', 'O'), ('T', 'T'), ('<START>', 'I'), ('<START>', 'L'), ('<START>', 'O'), ('<START>', 'P'), ('<START>', 'S'), ('<START>', 'T'), ('I', '<END>'), ('M', '<END>'), ('O', '<END>')]


In [27]:
allowed_transitions = sorted([('I', 'I'), ('M', 'O'), ('P', 'P'), ('T', 'T'), ('S', 'S'), ('O', 'O'), ('M', 'M'), ('S', 'O'), ('T', 'O'), ('L', 'L'), ('I', 'M'), ('M', 'I'), ('L', 'O'), ('P', 'O'), ('O', 'M')])
allowed_transitions

[('I', 'I'),
 ('I', 'M'),
 ('L', 'L'),
 ('L', 'O'),
 ('M', 'I'),
 ('M', 'M'),
 ('M', 'O'),
 ('O', 'M'),
 ('O', 'O'),
 ('P', 'O'),
 ('P', 'P'),
 ('S', 'O'),
 ('S', 'S'),
 ('T', 'O'),
 ('T', 'T')]

In [ ]:
len(transitions_list)

In [37]:
max(src.config.label_decoding.keys())

6

In [38]:
src.config.allowed_transitions_encoded

[(0, 0),
 (0, 2),
 (1, 1),
 (1, 3),
 (2, 0),
 (2, 2),
 (2, 3),
 (3, 2),
 (3, 3),
 (6, 3),
 (6, 6),
 (4, 3),
 (4, 4),
 (5, 3),
 (5, 5),
 (7, 0),
 (7, 1),
 (7, 3),
 (7, 6),
 (7, 4),
 (7, 5),
 (0, 8),
 (2, 8),
 (3, 8)]

In [8]:
# Compute Transitions for BIO scheme

---

In [40]:
inputs_ids = [149,19, 17, 13, 6,  16, 16, 13, 3,  16, 20, 14, 22, 13, 3,  7,  7,  4,  17, 13, 13, 20, 13, 14, 8,  3, 16,  9,  3,  13, 10, 19, 5,  4,  18, 22, 10, 17, 15, 19, 18, 16, 16, 20, 17, 4,  20, 13, 7,  20, 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  12,  0]
mask = [1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]
labels = [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1]
logits: torch.rand([3, 72, 7])

# print(*mask[1:-1], sep=',\t')
print(*inputs_ids[1:-1], sep=',\t')
print(*labels[:-2], sep=',\t')

19,	17,	13,	6,	16,	16,	13,	3,	16,	20,	14,	22,	13,	3,	7,	7,	4,	17,	13,	13,	20,	13,	14,	8,	3,	16,	9,	3,	13,	10,	19,	5,	4,	18,	22,	10,	17,	15,	19,	18,	16,	16,	20,	17,	4,	20,	13,	7,	20,	1,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	12
0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1,	-1


In [41]:
len(inputs_ids)

70

In [40]:
import pandas as pd

inputs0 = torch.tensor([149,  19,  17,  13,   6,  16,  16,  13,   3,  16,  20,  14,  22,  13,
           3,   7,   7,   4,  17,  13,  13,  20,  13,  14,   8,   3,  16,   9,
           3,  13,  10,  19,   5,   4,  18,  22,  10,  17,  15,  19,  18,  16,
          16,  20,  17,   4,  20,  13,   7,  20,   1,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0])
mask0 = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
labels0 = torch.tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100])

# inputs1 = [149,19, 11, 11, 4,  17, 8,  7,  9,  4,  4,  15, 6,  18, 10, 3,  16, 10, 22, 17, 13, 17, 5,  17, 13, 12, 5,  10, 17, 8,  13, 8,  8,  10, 13, 10, 11, 5,  16, 5,  12, 12, 11, 10, 6,  8,  4,  14, 8,  18, 4,  8,  10, 16, 4,  16, 10, 10, 5,  15, 10, 12, 18, 6,  14, 14, 11, 9,  5,  8,  7,  1]

inputs2 = torch.tensor([149,  19,   7,   7,  11,   4,  18,   8,   8,  16,   4,   4,  14,   4,
           4,   5,  19,   7,   6,   4,   5,  11,   7,  15,   7,   7,  22,   6,
          11,   7,  13,   3,   8,   3,  14,  11,   6,  17,  21,   5,  18,  12,
           5,  14,   6,   5,  13,   9,  20,  21,   5,   9,   4,   7,  13,  10,
          15,   3,   4,  22,  16,  12,   5,   8,  14,  16,  11,  13,  12,  10,
           4,   1])
mask2 = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
labels2 = torch.tensor([   5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
            5,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
            3,    3,    3,    3,    3,    3,    3,    3,    3,    3, -100, -100])

# print(('{:>4}'*len(inputs0)).format(*inputs0))
# print()

labels0 = torch.where(labels0 == -100, torch.tensor(-1), labels0)
labels2 = torch.where(labels2 == -100, torch.tensor(-1), labels2)


inputs0 = inputs0[1:-1]
inputs2 = inputs2[1:-1]

mask0 = mask0[1:-1]
mask2 = mask2[1:-1]

labels0 = labels0[:-2]
labels2 = labels2[:-2]

inputs0 = inputs0 * (labels0 != -1)
inputs2 = inputs2 * (labels2 != -1)

mask0 = mask0 * (labels0 != -1)
mask2 = mask2 * (labels2 != -1)



print(('{:>4}'*len(inputs0)).format(*inputs0))
print(('{:>4}'*len(mask0)).format(*mask0))
print(('{:>4}'*len(labels0)).format(*labels0))
print()
print(('{:>4}'*len(inputs2)).format(*inputs2))
print(('{:>4}'*len(mask2)).format(*mask2))
print(('{:>4}'*len(labels2)).format(*labels2))
# print('{:>2}'.format(*Mask))


  19  17  13   6  16  16  13   3  16  20  14  22  13   3   7   7   4  17  13  13  20  13  14   8   3  16   9   3  13  10  19   5   4  18  22  10  17  15  19  18  16  16  20  17   4  20  13   7  20   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

  19   7   7  11   4  18   8   8  16   4   4  14   4   4   5  19   7   6   4   5  11   7  15   7   7  22   6  11   7  13   3   8   3  14  11   6  17  21   5

In [38]:
(labels0 == -1).sum()

tensor(21)

In [ ]:
# Calculate the number of last items to set to 0
num_zeros = (mask0 == 0).sum()

# Set the last num_zeros items of inputs0 to 0
inputs0[-num_zeros:] = 0